# Homework 1c Part 3: SQL

To start with, if you felt the class was unclear, check out the following tutorial: https://mode.com/sql-tutorial/introduction-to-sql/

Now! We'll be using sqlite to access a database. Start by downloading the sql lite file and putting it in the same directory as this notebook: https://www.kaggle.com/datasets/kaggle/sf-salaries (hit the 'download' button in the upper right). Check out the description of the data so you know the table / column names.

The following code will use sqlite to create a database connection.

In [95]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("database.sqlite")
crsr = conn.cursor()

Before we proceed, please note that every task **must be completed using a single SQL query**, unless mentioned otherwise or given as prompt.     
(e.g. using print statements are fine, but you should not be using pandas library to work with the dataset.)

# Exploration

Problem 1:

Try to create a query that gives you a data frame of the EmployeeName, JobTitle, and BasePay from the salaries table.

In [96]:
query = 'SELECT EmployeeName, JobTitle, BasePay FROM salaries'

df = pd.read_sql(query, conn)
df.head()

,EmployeeName,JobTitle,BasePay
0,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411.18
1,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),155966.02
2,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),212739.13
3,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,77916
4,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",134401.6


Problem 2.

Modify your query from Problem 1 to limit it to the year 2012.

In [97]:
query = 'SELECT EmployeeName, JobTitle, BasePay FROM salaries WHERE Year = 2012'

df = pd.read_sql(query, conn)
df.head()

,EmployeeName,JobTitle,BasePay
0,Gary Altenberg,"Lieutenant, Fire Suppression",128808.87
1,Gregory Suhr,Chief of Police,302578.00
2,Khoa Trinh,Electronic Maintenance Tech,111921.00
3,Joanne Hayes-White,"Chief, Fire Department",296943.01
4,Frederick Binkley,EMT/Paramedic/Firefighter,126863.19


Problem 3:

Further limit the table to the year 2012, employees making under 150,000, and sort in descending order by salary.

In [98]:
query = 'SELECT EmployeeName, JobTitle, BasePay FROM salaries WHERE Year = 2012 AND TotalPay <= 150000 ORDER BY TotalPay DESC'

df = pd.read_sql(query, conn)
df.head()

,EmployeeName,JobTitle,BasePay
0,Kevin Salas,Firefighter,110847.11
1,Delene Wolf,Dept Head I,146497.50
2,Steven Stocker,Sergeant 3,135977.79
3,Wayne Yu,Electronic Maintenance Tech,101088.00
4,Eric Altorfer,Sergeant 2,129175.75


# Aggregation

Problem 4:

Get the average base pay from the table.

In [99]:
query = 'SELECT AVG(BasePay) FROM salaries'

df = pd.read_sql(query, conn)
df.head()

,AVG(BasePay)
0,66053.729288


Problem 5:

Produce and print the head of a dataframe that shows the average pay for each year (only use a single, simple query). Your result should have a column for the year and a column for the average base pay.

In [100]:
query = 'SELECT Year, AVG(BasePay) FROM salaries GROUP BY Year'

df = pd.read_sql(query, conn)
df.head()

,Year,AVG(BasePay)
0,2011,63595.956517
1,2012,65436.406857
2,2013,68509.832156
3,2014,66557.437750


Problem 6:

Create a dataframe with averages of base pay, benefits, and overtime for each job title, as well as a column with the average of these three values.

In [101]:
query = 'SELECT JobTitle, AVG(BasePay), AVG(Benefits), AVG(OvertimePay), (AVG(BasePay) + AVG(Benefits) + AVG(OvertimePay)) AS Average FROM salaries GROUP BY JobTitle'
df = pd.read_sql(query, conn)
#df.head()

# Table Creation

Problem 7:

Now we'll create our own table in our database. Separate the Salaries table by Year, and add it back to the database. (You may use basic python to complete the task, however, should still use SQL to query the data.)

In [103]:
for y in ['2011','2012','2013','2014']:
    query = f'''CREATE TABLE IF NOT EXISTS Y{y} AS SELECT * FROM salaries WHERE Year = {y}'''

    df.to_sql(name='Y'+y, con=conn, if_exists='replace')
    crsr.execute(query)
    print(f'Created table Y{y}')

conn.commit()

Created table Y2011
Created table Y2012
Created table Y2013
Created table Y2014


In [106]:
# TESTING
# query = 'SELECT * FROM Y2011'
# new_df = pd.read_sql(query, conn)
# new_df.head()

,index,JobTitle,AVG(BasePay),AVG(Benefits),AVG(OvertimePay),Average
0,0,ACCOUNT CLERK,43300.806506,0.00,373.200843,43674.007349
1,1,ACCOUNTANT,46643.172000,0.00,0.000000,46643.172000
2,2,ACCOUNTANT INTERN,28732.663958,0.00,24.430625,28757.094583
3,3,"ACPO,JuvP, Juv Prob (SFERS)",62290.780000,17975.59,0.000000,80266.370000
4,4,ACUPUNCTURIST,66374.400000,0.00,0.000000,66374.400000


# Table Joining

Problem 8:

We'll move on to a new dataset for the next steps.
Download the dataset from here (https://www.kaggle.com/datasets/luizpaulodeoliveira/imdb-project-sql) and load the sqlite file same as before. Start by just selecting everything in the "movies" table to see what it looks like.

In [107]:
conn = sqlite3.connect("movies.sqlite")
query = 'select * from movies'

df = pd.read_sql(query, conn)
df.head()

,id,original_title,budget,popularity,release_date,revenue,title,vote_average,vote_count,overview,tagline,uid,director_id
0,43597,Avatar,237000000,150,2009-12-10,2787965087,Avatar,7.2,11800,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,19995,4762
1,43598,Pirates of the Caribbean: At World's End,300000000,139,2007-05-19,961000000,Pirates of the Caribbean: At World's End,6.9,4500,"Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins.",285,4763
2,43599,Spectre,245000000,107,2015-10-26,880674609,Spectre,6.3,4466,A cryptic message from Bond’s past sends him o...,A Plan No One Escapes,206647,4764
3,43600,The Dark Knight Rises,250000000,112,2012-07-16,1084939099,The Dark Knight Rises,7.6,9106,Following the death of District Attorney Harve...,The Legend Ends,49026,4765
4,43601,John Carter,260000000,43,2012-03-07,284139100,John Carter,6.1,2124,"John Carter is a war-weary, former military ca...","Lost in our world, found in another.",49529,4766


Problem 9:

Create a dataframe that includes the entire contents of "movies" table as well as the director's name.

In [114]:
query = f'''
  SELECT m.*, d.name AS director_name 
  FROM movies AS m 
  JOIN directors AS d ON m.director_id = d.id
  '''
df = pd.read_sql(query, conn)

print(df.head(5))
df.size

      id                            original_title     budget  popularity  \
0  43597                                    Avatar  237000000         150   
1  43598  Pirates of the Caribbean: At World's End  300000000         139   
2  43599                                   Spectre  245000000         107   
3  43600                     The Dark Knight Rises  250000000         112   
4  43601                               John Carter  260000000          43   

  release_date     revenue                                     title  \
0   2009-12-10  2787965087                                    Avatar   
1   2007-05-19   961000000  Pirates of the Caribbean: At World's End   
2   2015-10-26   880674609                                   Spectre   
3   2012-07-16  1084939099                     The Dark Knight Rises   
4   2012-03-07   284139100                               John Carter   

   vote_average  vote_count  \
0           7.2       11800   
1           6.9        4500   
2          

66822

# Analysis

The next few problems will be more involved! You'll need to combine some concepts you've learned. For each cell, show your work.
\
Remember, the answers should be in **a single query**.

\
Problem 10:

What is the average budget used for the top 10 grossing movies?


In [125]:
query = '''
SELECT AVG(budget) AS average_budget
FROM (
  SELECT budget FROM movies
  ORDER BY revenue DESC
  LIMIT 10
)
'''
df = pd.read_sql(query, conn)
df["average_budget"][0]

195100000.0

Problem 11:

Which directors have the highest overall voting average? - show the top 5 directors' name and their average rating

In [126]:
query = '''
SELECT d.name AS director_name, AVG(m.vote_average) AS average_rating
FROM movies AS m
JOIN directors AS d ON m.director_id = d.id
GROUP BY director_name
ORDER BY average_rating DESC
LIMIT 5
'''
df = pd.read_sql(query, conn)
df


,director_name,average_rating
0,Gary Sinyor,10.00
1,Rohit Jugraj,9.50
2,Lance Hool,9.30
3,Floyd Mutrux,8.50
4,Tim McCanlies,8.45


Problem 12:

What are the top five directors by how much their average budget is?

In [135]:
query = '''
SELECT d.name AS director_name, AVG(m.budget) AS average_budget
FROM movies AS m
JOIN directors AS d ON m.director_id = d.id
GROUP BY director_name
ORDER BY average_budget DESC
LIMIT 5
'''
df = pd.read_sql(query, conn)
df

,director_name,average_budget
0,Byron Howard,2.600000e+08
1,Lee Unkrich,2.000000e+08
2,Dan Scanlon,2.000000e+08
3,David Yates,1.933333e+08
4,Brenda Chapman,1.850000e+08
